<a href="https://colab.research.google.com/github/tjhung23/MBC/blob/main/03_%C4%90%E1%BB%91i_So%C3%A1t_Momo_%2B_Zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xlsxwriter

In [ ]:

import glob
import os
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import getpass
import pytz
import time
# --------------------------------------------------------------------------------------------------------------------------
vn_tz = pytz.timezone('Asia/Ho_Chi_Minh')
now = datetime.now(vn_tz)


user = getpass.getuser()
df_chiet_khau = pd.DataFrame({
    'Chiết khấu tạm tính xuất hóa đơn': ['D5M', 'D15M', 'D30M', 'WIN69XM', 'WIN89XM'],
    'Chiết khấu mức dưới 1.3 tỷ': [20, 20, 20, 15, 15],
    'Loại': ['Ngày', 'Ngày', 'Ngày', 'Tháng', 'Tháng'],
    'Đơn giá bao gồm VAT': [5000, 15000, 30000, 69000, 89000]
})
#---------------------------------------------------------------------------------------------------------------------------
xlpath = '/content/drive/My Drive/KeToan_HachToanGiaoDich/Momo/'
result = glob.glob(xlpath + "*.*")
for filename in result:
  # try:
    if filename.endswith((".xlsx", ".xls")):
      df = pd.read_excel(filename, dtype=str)
#---------------------------------------------------------------------------------------------------------------------------
      #DWH_doi soat CTT

      if df.columns[0] == 'Kênh':
        dwh = df.copy()
        dwh['Ngày thanh toán'] =dwh['Ngày thanh toán'].astype(str).str.split(" ").str[0]
        dwh[['Year', 'Month', 'Day']] = dwh['Ngày thanh toán'].str.split('-', expand=True)
        dwh['Ngày thanh toán'] = dwh['Day'] + '-' + dwh['Month'] + '-' + dwh['Year']
        dwh = dwh.drop(['Year','Day'], axis=1)
        dwh = dwh.rename(columns={'Month': 'Tháng thanh toán'})
        cols = ['Giá gói','Giảm giá gói','Tổng tiền thanh toán']
        dwh[cols] = dwh[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

        # Tạo mapping dictionary từ df_chiet_khau
        goi_to_loai = df_chiet_khau.set_index('Chiết khấu tạm tính xuất hóa đơn')['Loại'].to_dict()


        goi_to_ngay = df_chiet_khau.set_index('Chiết khấu tạm tính xuất hóa đơn')['Chiết khấu mức dưới 1.3 tỷ'].to_dict()

        # Map vào dwh để tạo cột 'Loại'
        dwh['Loại GC'] = dwh['Gói cước'].map(goi_to_loai)

        dwh['Giá gói chưa VAT'] = dwh['Tổng tiền thanh toán']/1.1

        dwh['Mức chiết khấu dưới 1.3 tỷ'] = dwh['Gói cước'].map(goi_to_ngay)

        dwh['Ngày thanh toán'] = dwh['Ngày thanh toán'].astype(str).str.split(" ").str[0]

        dwh['Ngày thanh toán'] = pd.to_datetime(dwh['Ngày thanh toán'], dayfirst=True, errors='coerce')

        # Tạo cột 'Ngày hạch toán' = Ngày thanh toán + 1 ngày
        dwh['Ngày hạch toán'] = dwh['Ngày thanh toán'] + pd.Timedelta(days=1)

        dwh['Tháng hạch toán'] = dwh['Ngày hạch toán'].dt.month
      if df.columns[0] == 'Mã kênh':
        dwh_bhdl = df.copy()
        dwh_bhdl = dwh_bhdl[dwh_bhdl['Tên kênh'].isin(['Momo Etopup'])]
        cols_to_convert = ['Giá gói', 'Giá sim', 'Phí vận chuyển','Giá kit']  # thay bằng danh sách cột bạn muốn chuyển

        dwh_bhdl[cols_to_convert] = dwh_bhdl[cols_to_convert].apply(pd.to_numeric, errors='coerce')

        dwh_bhdl = dwh_bhdl[dwh_bhdl['Tên kênh'].isin(['Momo Etopup'])]
        dwh_bhdl = dwh_bhdl[dwh_bhdl['Loại số'].isin(['Số thường', 'Số đẹp'])]
        dwh_bhdl['Giá gói chưa VAT'] = dwh_bhdl['Giá gói']/1.1
        dwh_bhdl['Giá HM chưa VAT'] = np.where(dwh_bhdl['Giá sim'] <25000, dwh_bhdl['Giá sim']/1.1, 25000/1.1)
        dwh_bhdl['Giá sim chưa VAT'] = np.where(dwh_bhdl['Giá sim'] <25000, 0, 25000/1.1)
        dwh_bhdl['Phí ship chưa VAT'] = (dwh_bhdl['Phí vận chuyển']/1.1)
        dwh_bhdl['Chiết khấu bộ kit'] = 20000/1.1
        dwh_bhdl['Chiết khấu gói cước'] = 10000/1.1
        # dwh_bhdl['Ngày tạo đơn'] = dwh_bhdl['Ngày tạo đơn'].astype(str).str.split(" ").str[0]

        # dwh_bhdl['Ngày tạo đơn'] = pd.to_datetime(dwh_bhdl['Ngày tạo đơn'], dayfirst=True, errors='coerce')




In [ ]:
# Tạo cột "Tiền gói - Data" nếu "Gói cước" bắt đầu bằng "D"
dwh['Tiền gói - Data'] = np.where(dwh['Gói cước'].str.startswith('D'), dwh['Giá gói chưa VAT'], 0)

# Tạo cột "Tiền gói - Combo 1 tháng" nếu "Gói cước" bắt đầu bằng "W"
dwh['Tiền gói - Combo 1 tháng'] = np.where(dwh['Gói cước'].str.startswith('W'), dwh['Giá gói chưa VAT'], 0)
dwh['SDT'] = dwh['SDT mua']
dwh['Ngày thanh toán'] = dwh['Ngày thanh toán'].astype(str).str.split(" ").str[0]
dwh['Date'] = dwh['Ngày thanh toán']

In [ ]:
dwh_bhdl['Tiền gói - Combo 1 tháng'] = dwh_bhdl['Giá gói chưa VAT']
dwh_bhdl['Tiền gói - Data'] = 0
dwh_bhdl['SDT'] = dwh_bhdl['SĐT đặt']
dwh_bhdl['Ngày tạo đơn'] = dwh_bhdl['Ngày tạo đơn'].astype(str).str.split(" ").str[0]
dwh_bhdl['Date'] = dwh_bhdl['Ngày tạo đơn']
#  dwh_bhdl['Ngày tạo đơn'] = pd.to_datetime(dwh_bhdl['Ngày tạo đơn'], dayfirst=True, errors='coerce')
dwh_bhdl['Gói cước'] = dwh_bhdl['Tên gói']

In [ ]:
# dwh_bhdl['Ngày tạo đơn'].unique()

In [ ]:
MOMO_MISA = dwh.groupby(['Ngày thanh toán', 'Gói cước']).agg(
    So_luong=('Gói cước', 'count'),                     # Đếm số dòng theo từng gói
    Tong_tien_thanh_toan=('Tổng tiền thanh toán', 'sum') # Tổng tiền thanh toán
).reset_index()
# MOMO_MISA['đ'] = dwh['Gói cước'].map(goi_to_loai)
MOMO_MISA['Tỷ lệ CK'] = MOMO_MISA['Gói cước'].map(goi_to_ngay)
MOMO_MISA = MOMO_MISA[MOMO_MISA['Gói cước'].isin(['D5M', 'D15M', 'D30M', 'WIN69XM', 'WIN89XM'])]
MOMO_MISA['Tiền chiết khấu'] = MOMO_MISA['Tong_tien_thanh_toan'] * MOMO_MISA['Tỷ lệ CK'] / 100
MOMO_MISA['Sau chiết Khấu'] = MOMO_MISA['Tong_tien_thanh_toan'] - MOMO_MISA['Tiền chiết khấu']
MOMO_MISA['Tiền CK/VAT'] = MOMO_MISA['Tiền chiết khấu'] / 1.1
MOMO_MISA['Thuế'] = (MOMO_MISA['Tong_tien_thanh_toan']/1.1 - MOMO_MISA['Tiền CK/VAT']) * 0.1
MOMO_MISA = MOMO_MISA.rename(columns={'So_luong': 'Số Lượng', 'Tong_tien_thanh_toan': 'Thành tiền'})
MOMO_MISA['Thành tiền/VAT'] = MOMO_MISA['Thành tiền'] / 1.1
MOMO_MISA = MOMO_MISA.groupby(["Ngày thanh toán","Gói cước"]).sum(numeric_only=True).reset_index()

In [ ]:
# Các cột giữ nguyên
id_vars = ['Ngày thanh toán', 'Gói cước','Số Lượng']

# Các cột muốn melt
value_vars = ['Thành tiền', 'Tiền chiết khấu', 'Thành tiền/VAT']

# Thực hiện melt
MOMO_MISA_melt = MOMO_MISA.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Type',
    value_name='Amount'
)


In [ ]:
MOMO_MISA_melt['Tài Khoản'] = np.where(MOMO_MISA_melt['Type'] == 'Thành tiền', '338731',
                                       np.where(MOMO_MISA_melt['Type'] == 'Tiền chiết khấu','52113',
                                       np.where(MOMO_MISA_melt['Type'] == 'Thành tiền/VAT','33311',"")))


In [ ]:
ZALO_MISA = dwh.groupby(['Ngày thanh toán', 'Gói cước','Giá gói']).agg(
    So_luong=('Gói cước', 'count'),                     # Đếm số dòng theo từng gói
    Tong_tien_thanh_toan=('Tổng tiền thanh toán', 'sum') # Tổng tiền thanh toán
).reset_index()
# ZALO_MISA['đ'] = dwh['Gói cước'].map(goi_to_loai)
ZALO_MISA['Tỷ lệ CK'] = ZALO_MISA['Gói cước'].map(goi_to_ngay)
ZALO_MISA = ZALO_MISA[ZALO_MISA['Gói cước'].isin(['D5Z', 'D15Z', 'D30Z'])]
ZALO_MISA['Tiền chiết khấu'] = ZALO_MISA['Tong_tien_thanh_toan'] * ZALO_MISA['Tỷ lệ CK'] / 100
ZALO_MISA['Sau chiết Khấu'] = ZALO_MISA['Tong_tien_thanh_toan'] - ZALO_MISA['Tiền chiết khấu']
ZALO_MISA['Tiền CK/VAT'] = ZALO_MISA['Tiền chiết khấu'] / 1.1
ZALO_MISA['Thuế'] = (ZALO_MISA['Tong_tien_thanh_toan']/1.1 - ZALO_MISA['Tiền CK/VAT']) * 0.1
ZALO_MISA = ZALO_MISA.rename(columns={'So_luong': 'Số Lượng', 'Tong_tien_thanh_toan': 'Thành tiền'})
ZALO_MISA['Thành tiền/VAT'] = ZALO_MISA['Thành tiền'] / 1.1

In [ ]:
# Các cột giữ nguyên
id_vars = ['Ngày thanh toán', 'Gói cước','Số Lượng']

# Các cột muốn melt
value_vars = ['Thành tiền', 'Tiền chiết khấu', 'Thành tiền/VAT']

# Thực hiện melt
ZALO_MISA_melt = ZALO_MISA.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Type',
    value_name='Amount'
)


In [ ]:
ZALO_MISA_melt['Tài Khoản'] = np.where(ZALO_MISA_melt['Type'] == 'Thành tiền', '338731',
                                       np.where(ZALO_MISA_melt['Type'] == 'Tiền chiết khấu','52113',
                                       np.where(ZALO_MISA_melt['Type'] == 'Thành tiền/VAT','33311',"")))

In [ ]:
MOMO_sothuong = dwh_bhdl[dwh_bhdl['Loại số'] == 'Số thường'].copy()
MOMO_sothuong = MOMO_sothuong.fillna(0)  # Xử lý NaN nếu cần
MOMO_sothuong = MOMO_sothuong.groupby(['Ngày tạo đơn', 'Loại sim (order)', 'Tên gói']).agg({
    'Loại sim (order)': 'count',  # Đếm số lượng
    'Giá kit': 'sum',
    'Giá gói': 'sum',
    'Giá sim': 'sum',
    'Phí vận chuyển': 'sum',
    'Giá HM chưa VAT': 'sum',
    'Giá sim chưa VAT': 'sum',
    'Phí ship chưa VAT': 'sum',
    'Giá gói chưa VAT': 'sum'
}).rename(columns={'Loại sim (order)': 'Số lượng'}).reset_index()

In [ ]:
# Thực hiện melt
MOMO_sothuong_melt = MOMO_sothuong.melt(
    id_vars=["Ngày tạo đơn","Loại sim (order)","Tên gói","Số lượng"],
    value_vars=['Giá HM chưa VAT', 'Giá sim chưa VAT', 'Phí ship chưa VAT', 'Giá gói chưa VAT'],
    var_name='Type',
    value_name='Amount'
)
MOMO_sothuong_melt['Amount'] = MOMO_sothuong_melt['Amount'].fillna(0).astype(int)
MOMO_sothuong_melt = MOMO_sothuong_melt.loc[MOMO_sothuong_melt['Amount'] != 0]

In [ ]:
MOMO_sothuong_melt['Tài khoản'] = np.where(MOMO_sothuong_melt['Type'] == 'Phí ship chưa VAT', '33875',
                                      np.where(MOMO_sothuong_melt['Type'] == 'Giá HM chưa VAT','338712',
                                      np.where(MOMO_sothuong_melt['Type'] == 'Giá sim chưa VAT','338711',
                                      np.where(MOMO_sothuong_melt['Type'] == 'Giá gói chưa VAT','3387241',""))))

In [ ]:
MOMO_sodep = dwh_bhdl[dwh_bhdl['Loại số'] == 'Số đẹp'].copy()
MOMO_sodep = MOMO_sodep.fillna(0)  # Xử lý NaN nếu cần
MOMO_sodep = MOMO_sodep.groupby(['Ngày tạo đơn', 'Loại sim (order)', 'Tên gói']).agg({
    'Loại sim (order)': 'count',  # Đếm số lượng
    'Giá kit': 'sum',
    'Giá gói': 'sum',
    'Giá sim': 'sum',
    'Phí vận chuyển': 'sum',
    'Giá HM chưa VAT': 'sum',
    'Giá sim chưa VAT': 'sum',
    'Phí ship chưa VAT': 'sum',
    'Giá gói chưa VAT': 'sum',
    'Chiết khấu bộ kit': 'sum',
    'Chiết khấu gói cước': 'sum'
}).rename(columns={'Loại sim (order)': 'Số lượng'}).reset_index()

In [ ]:
# Thực hiện melt
MOMO_sodep_melt = MOMO_sodep.melt(
    id_vars=["Ngày tạo đơn","Loại sim (order)","Tên gói","Số lượng"],
    value_vars=['Giá HM chưa VAT', 'Giá sim chưa VAT', 'Phí ship chưa VAT', 'Giá gói chưa VAT','Chiết khấu bộ kit','Chiết khấu gói cước'],
    var_name='Type',
    value_name='Amount'
)
MOMO_sodep_melt['Amount'] = MOMO_sodep_melt['Amount'].fillna(0).astype(int)
MOMO_sodep_melt = MOMO_sodep_melt.loc[MOMO_sodep_melt['Amount'] != 0]

In [ ]:
MOMO_sodep_melt['Tài khoản'] = np.where(MOMO_sodep_melt['Type'] == 'Phí ship chưa VAT', '33875',
                                      np.where(MOMO_sodep_melt['Type'] == 'Giá HM chưa VAT','338712',
                                      np.where(MOMO_sodep_melt['Type'] == 'Giá sim chưa VAT','338711',
                                      np.where(MOMO_sodep_melt['Type'] == 'Giá gói chưa VAT','3387241',
                                      np.where(MOMO_sodep_melt['Type'] == 'Chiết khấu bộ kit','521111',
                                      np.where(MOMO_sodep_melt['Type'] == 'Chiết khấu gói cước', '52113',""))))))

In [ ]:
MOMO_banso = pd.concat([MOMO_sothuong_melt, MOMO_sodep_melt])
# MOMO_banso

In [ ]:
# Tạo timestamp
timestamp = now.strftime("%Y-%m-%d_%H-%M")
with pd.ExcelWriter(f'/content/drive/My Drive/Hung/test/Doi Soat MOMO - {timestamp}.xlsx', engine='xlsxwriter') as writer:
    dwh.to_excel(writer, sheet_name='DWH', index=False)
    dwh_bhdl.to_excel(writer, sheet_name='DWH_bhdl', index=False)
    MOMO_MISA.to_excel(writer, sheet_name='MOMO_Bán gói', index=False)
    # MOMO_MISA2.to_excel(writer, sheet_name='MOMO_MISA', index=False)
    MOMO_sothuong.to_excel(writer, sheet_name='MOMO_Số thường', index=False)
    MOMO_sodep.to_excel(writer, sheet_name='MOMO_Số đẹp', index=False)
    ZALO_MISA.to_excel(writer, sheet_name='ZALO_Bán gói', index=False)
    # ZALO_MISA2.to_excel(writer, sheet_name='ZALO_MISA', index=False)

In [ ]:
# # lấy số điện thoại KYC và mua các gói từ MOMO,ZALO
# # List of your DataFrames and their names
# dfs = [dwh, dwh_bhdl]
# names = ['dwh', 'dwh_bhdl']

# # Columns to select in the output
# columns_to_select = [
#     'Date',
#     'SDT',
#     'KYC',
#     'Gói cước',
#     'Tiền gói - Combo 1 tháng',
#     'Tiền gói - Data'
# ]

# # Columns to filter (check for non-zero values)
# columns_to_filter = [
#     'Tiền gói - Combo 1 tháng',
#     'Tiền gói - Data'
# ]

# # Initialize an empty list to store filtered DataFrames
# filtered_dfs = []

# # Loop through each DataFrame
# for i, df in enumerate(dfs):
#     # Ensure all filter columns exist in the DataFrame
#     available_filter_columns = [col for col in columns_to_filter if col in df.columns]

#     # Create a condition to filter rows where any of the specified columns is not 0
#     condition = False
#     for col in available_filter_columns:
#         # Convert column to numeric, handling non-numeric values
#         df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
#         condition |= (df[col] != 0)

#     # Ensure all select columns exist in the DataFrame
#     available_select_columns = [col for col in columns_to_select if col in df.columns]

#     # Filter rows and select specified columns
#     filtered = df[condition][available_select_columns].copy()

#     # Add a 'source' column with the DataFrame name
#     filtered['source'] = names[i]

#     # Append filtered DataFrame to the list
#     filtered_dfs.append(filtered)

#     # Print the number of rows found
#     print(f"DataFrame {names[i]}: Found {len(filtered)} rows where any specified column != 0")

# # Combine all filtered DataFrames into one
# result_dfmomo = pd.concat(filtered_dfs, ignore_index=True)

# # Handle NaN in SDT and KYC
# result_dfmomo = result_dfmomo.fillna({'SDT': '', 'KYC': ''})
# result_dfmomo = result_dfmomo.groupby(["Date","Gói cước"]).sum(numeric_only=True).reset_index()